# Ilist : structure analysis

## Goals

- understand the structure of Ilist object
- introduce methods for managing structure


-----

## Example

The reuse Ilist is :

<img src="https://loco-philippe.github.io/ES/ilist_merge.png" width="700">


In [1]:
import os
os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from ilist import Ilist
from pprint import pprint

img = Ilist.from_file('score.il')                # reuse Ilist from aggregation Notebook

## relationship

the relationships between Iindex can be shown with the img Ilist

<img src="https://loco-philippe.github.io/ES/ilist_canonical.png" width="600">


In [2]:
print('primary Iindex : ',img.primary, [img.idxname[i] for i in img.primary])     # three primary Iindex
print('dimension : ', img.dimension, '\n')

print('\ncoupling infos  :')
pprint(img.indexinfos(keys=['num', 'name', 'parent','pparent', 'pname', 'typecoupl']), width=120)  

print('\ncoupling rate for primary Iindex :')
inforate = img.indexinfos(keys=['name', 'linkrate', 'typecoupl'], base=True)
pprint([inf for inf in inforate if inf['typecoupl'] in ['crossed', 'linked']], width=120)

print('\nactual length :', len(img), 'and length if primary Iindex are crossed :',  img.lencomplete)

primary Iindex :  [0, 5, 7] ['full name', 'course', 'examen']
dimension :  3 


coupling infos  :
[{'name': 'full name', 'num': 0, 'parent': 0, 'pname': 'full name', 'pparent': 0, 'typecoupl': 'crossed'},
 {'name': 'last name', 'num': 1, 'parent': 0, 'pname': 'full name', 'pparent': 0, 'typecoupl': 'derived'},
 {'name': 'first name', 'num': 2, 'parent': 0, 'pname': 'full name', 'pparent': 0, 'typecoupl': 'derived'},
 {'name': 'surname', 'num': 3, 'parent': 0, 'pname': 'full name', 'pparent': 0, 'typecoupl': 'coupled'},
 {'name': 'group', 'num': 4, 'parent': 0, 'pname': 'full name', 'pparent': 0, 'typecoupl': 'derived'},
 {'name': 'course', 'num': 5, 'parent': 1, 'pname': 'last name', 'pparent': 5, 'typecoupl': 'linked'},
 {'name': 'year', 'num': 6, 'parent': 6, 'pname': 'year', 'pparent': -1, 'typecoupl': 'unique'},
 {'name': 'examen', 'num': 7, 'parent': 1, 'pname': 'last name', 'pparent': 7, 'typecoupl': 'linked'}]

coupling rate for primary Iindex :
[{'linkrate': 1.0, 'name': 'full 

## Transformation to complete Ilist

When primary Iindex are crossed, an Ilist is complete (can be converted into matrix or in file with low size).

In [3]:
img.full(fillvalue=float('nan'))                 # convert linked indexes to crossed indexes

print('\ncoupling rate for primary Iindex :')
inforate = img.indexinfos(keys=['name', 'typecoupl'], base=True)
pprint([inf for inf in inforate if inf['typecoupl'] in ['crossed', 'linked']], width=120)

print('\nlength :', len(img), 'dimension :', img.dimension)


coupling rate for primary Iindex :
[{'name': 'full name', 'typecoupl': 'crossed'},
 {'name': 'course', 'typecoupl': 'crossed'},
 {'name': 'examen', 'typecoupl': 'crossed'}]

length : 36 dimension : 3


## Indexed matrix

With codec ou values ajustement, an Ilist can be transformed in a matrix with choosen dimension

In [8]:
import xarray
print(img.to_xarray())
print('\nObject img is complete and have the canonical order ? ', img.complete, img.iscanonorder())


<xarray.DataArray ['score'] (full name: 4, course: 3, examen: 3)>
array([[[nan, nan,  8.],
        [15., nan, nan],
        [nan, nan, nan]],

       [[ 6., nan, nan],
        [nan, nan, nan],
        [nan, 18., nan]],

       [[ 2., nan,  4.],
        [nan, nan, nan],
        [nan, 17., 18.]],

       [[nan, 12., 10.],
        [11., 15., 13.],
        [nan, nan, nan]]])
Coordinates:
  * full name   (full name) <U14 'philippe white' ... 'anne white'
  * course      (course) <U8 'english' 'math' 'software'
  * examen      (examen) <U2 't1' 't3' 't2'
    last name   (full name) <U5 'black' 'white' 'red' 'white'
    first name  (full name) <U8 'camille' 'philippe' 'anne' 'philippe'
    surname     (full name) <U10 'heisenberg' 'gus' 'saul' 'skyler'
    group       (full name) <U3 'gr2' 'gr1' 'gr3' 'gr3'
Attributes:
    year:     2021

Object img is complete and have the canonical order ?  True True


In [10]:
print('codec is not necessary in json object when Ilist is complete :\n', img.to_obj())   
print('\nconversion is reversible ? ', img.from_obj(img.to_obj()) == img)


codec is not necessary in json object when Ilist is complete :
 [['full name', ['philippe white', 'philippe black', 'camille red', 'anne white']], ['course', ['english', 'math', 'software']], ['examen', ['t1', 't3', 't2']], ['last name', ['white', 'black', 'red'], [0, [0, 1, 2, 0]]], ['first name', ['philippe', 'camille', 'anne'], [0, [0, 0, 1, 2]]], ['surname', ['heisenberg', 'gus', 'saul', 'skyler'], 0], ['group', ['gr1', 'gr2', 'gr3'], [0, [1, 2, 2, 0]]], ['year', [2021]], ['score', [nan, nan, 8, 15, nan, nan, nan, nan, nan, 6, nan, nan, nan, nan, nan, nan, 18, nan, 2, nan, 4, nan, nan, nan, nan, 17, 18, nan, 12, 10, 11, 15, 13, nan, nan, nan], -1]]

conversion is reversible ?  True


In [11]:
# matrix with dimension 2
img.nindex('course').coupling(img.nindex('examen'))   # transform two linked indexes in two derived or coupled indexes
print('new dimension : ', img.dimension, '\n')
print(img.to_xarray())

new dimension :  2 

<xarray.DataArray ['score'] (full name: 4, course: 9)>
array([[nan, nan, nan, nan, nan, nan,  8., nan, 15.],
       [nan, nan, 18.,  6., nan, nan, nan, nan, nan],
       [nan, nan, 17.,  2., nan, nan,  4., 18., nan],
       [12., 13., nan, nan, 15., nan, 10., nan, 11.]])
Coordinates:
  * full name   (full name) <U14 'philippe white' ... 'anne white'
  * course      (course) <U8 'english' 'math' 'software' ... 'software' 'math'
    examen      (course) <U2 't3' 't2' 't2' 't1' 't3' 't2' 't1' 't3' 't1'
    last name   (full name) <U5 'black' 'white' 'red' 'white'
    first name  (full name) <U8 'camille' 'philippe' 'anne' 'philippe'
    surname     (full name) <U10 'heisenberg' 'gus' 'saul' 'skyler'
    group       (full name) <U3 'gr2' 'gr1' 'gr3' 'gr3'
Attributes:
    year:     2021
